[Reference](https://towardsdatascience.com/paper-walkthrough-vision-transformer-vit-c5dcf76f1a7a)

In [22]:
pip install torchinfo

In [23]:
# Codeblock 1
import torch
import torch.nn as nn
from torchinfo import summary

In [3]:
# Codeblock 2
#(1)
BATCH_SIZE   = 1
IMAGE_SIZE   = 224
IN_CHANNELS  = 3

#(2)
PATCH_SIZE   = 16
NUM_HEADS    = 12
NUM_ENCODERS = 12
EMBED_DIM    = 768
MLP_SIZE     = EMBED_DIM * 4    # 768*4 = 3072

#(3)
NUM_PATCHES  = (IMAGE_SIZE//PATCH_SIZE) ** 2    # (224//16)**2 = 196

#(4)
DROPOUT_RATE = 0.1
NUM_CLASSES  = 10

In [4]:
# Codeblock 3
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
# Codeblock 4
class PatcherUnfold(nn.Module):
    def __init__(self):
        super().__init__()
        self.unfold = nn.Unfold(kernel_size=PATCH_SIZE, stride=PATCH_SIZE)    #(1)
        self.linear_projection = nn.Linear(in_features=IN_CHANNELS*PATCH_SIZE*PATCH_SIZE,
                                           out_features=EMBED_DIM)    #(2)
    # Codeblock 5
    def forward(self, x):
        print(f'original\t: {x.size()}')

        x = self.unfold(x)
        print(f'after unfold\t: {x.size()}')

        x = x.permute(0, 2, 1)    #(1)
        print(f'after permute\t: {x.size()}')

        x = self.linear_projection(x)
        print(f'after lin proj\t: {x.size()}')

        return x

In [7]:
# Codeblock 6
patcher_unfold = PatcherUnfold()
x = torch.randn(1, 3, 224, 224)
x = patcher_unfold(x)

original	: torch.Size([1, 3, 224, 224])
after unfold	: torch.Size([1, 768, 196])
after permute	: torch.Size([1, 196, 768])
after lin proj	: torch.Size([1, 196, 768])


In [8]:
# Codeblock 7
class PatcherConv(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=IN_CHANNELS,
                              out_channels=EMBED_DIM,
                              kernel_size=PATCH_SIZE,
                              stride=PATCH_SIZE)

        self.flatten = nn.Flatten(start_dim=2)

    def forward(self, x):
        print(f'original\t\t: {x.size()}')

        x = self.conv(x)    #(1)
        print(f'after conv\t\t: {x.size()}')

        x = self.flatten(x)    #(2)
        print(f'after flatten\t\t: {x.size()}')

        x = x.permute(0, 2, 1)    #(3)
        print(f'after permute\t\t: {x.size()}')

        return x

In [9]:
# Codeblock 8
patcher_conv = PatcherConv()
x = torch.randn(1, 3, 224, 224)
x = patcher_conv(x)

original		: torch.Size([1, 3, 224, 224])
after conv		: torch.Size([1, 768, 14, 14])
after flatten		: torch.Size([1, 768, 196])
after permute		: torch.Size([1, 196, 768])


In [11]:
# Codeblock 9
class PosEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.class_token = nn.Parameter(torch.randn(size=(BATCH_SIZE, 1, EMBED_DIM)),
                                        requires_grad=True)    #(1)
        self.pos_embedding = nn.Parameter(torch.randn(size=(BATCH_SIZE, NUM_PATCHES+1, EMBED_DIM)),
                                          requires_grad=True)    #(2)
        self.dropout = nn.Dropout(p=DROPOUT_RATE)  #(3)
    # Codeblock 10
    def forward(self, x):

        class_token = self.class_token
        print(f'class_token dim\t\t: {class_token.size()}')

        print(f'before concat\t\t: {x.size()}')
        x = torch.cat([class_token, x], dim=1)    #(1)
        print(f'after concat\t\t: {x.size()}')

        x = self.pos_embedding + x    #(2)
        print(f'after pos_embedding\t: {x.size()}')

        x = self.dropout(x)    #(3)
        print(f'after dropout\t\t: {x.size()}')

        return x

In [12]:
# Codeblock 11
pos_embedding = PosEmbedding()
x = pos_embedding(x)

class_token dim		: torch.Size([1, 1, 768])
before concat		: torch.Size([1, 196, 768])
after concat		: torch.Size([1, 197, 768])
after pos_embedding	: torch.Size([1, 197, 768])
after dropout		: torch.Size([1, 197, 768])


In [14]:
# Codeblock 12
class TransformerEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.norm_0 = nn.LayerNorm(EMBED_DIM)    #(1)

        self.multihead_attention = nn.MultiheadAttention(EMBED_DIM,    #(2)
                                                         num_heads=NUM_HEADS,
                                                         batch_first=True,
                                                         dropout=DROPOUT_RATE)

        self.norm_1 = nn.LayerNorm(EMBED_DIM)    #(3)

        self.mlp = nn.Sequential(    #(4)
            nn.Linear(in_features=EMBED_DIM, out_features=MLP_SIZE),    #(5)
            nn.GELU(),
            nn.Dropout(p=DROPOUT_RATE),
            nn.Linear(in_features=MLP_SIZE, out_features=EMBED_DIM),    #(6)
            nn.Dropout(p=DROPOUT_RATE)
        )

    # Codeblock 13
    def forward(self, x):

        residual = x    #(1)
        print(f'residual dim\t\t: {residual.size()}')

        x = self.norm_0(x)    #(2)
        print(f'after norm\t\t: {x.size()}')

        x = self.multihead_attention(x, x, x)[0]    #(3)
        print(f'after attention\t\t: {x.size()}')

        x = x + residual    #(4)
        print(f'after addition\t\t: {x.size()}')

        residual = x    #(5)
        print(f'residual dim\t\t: {residual.size()}')

        x = self.norm_1(x)    #(6)
        print(f'after norm\t\t: {x.size()}')

        x = self.mlp(x)    #(7)
        print(f'after mlp\t\t: {x.size()}')

        x = x + residual    #(8)
        print(f'after addition\t\t: {x.size()}')

        return x

In [15]:
# Codeblock 14
transformer_encoder = TransformerEncoder()
x = transformer_encoder(x)

residual dim		: torch.Size([1, 197, 768])
after norm		: torch.Size([1, 197, 768])
after attention		: torch.Size([1, 197, 768])
after addition		: torch.Size([1, 197, 768])
residual dim		: torch.Size([1, 197, 768])
after norm		: torch.Size([1, 197, 768])
after mlp		: torch.Size([1, 197, 768])
after addition		: torch.Size([1, 197, 768])


In [16]:
# Codeblock 15
class MLPHead(nn.Module):
    def __init__(self):
        super().__init__()

        self.norm = nn.LayerNorm(EMBED_DIM)
        self.linear_0 = nn.Linear(in_features=EMBED_DIM,
                                  out_features=EMBED_DIM)
        self.gelu = nn.GELU()
        self.linear_1 = nn.Linear(in_features=EMBED_DIM,
                                  out_features=NUM_CLASSES)    #(1)

    def forward(self, x):
        print(f'original\t\t: {x.size()}')

        x = self.norm(x)
        print(f'after norm\t\t: {x.size()}')

        x = self.linear_0(x)
        print(f'after layer_0 mlp\t: {x.size()}')

        x = self.gelu(x)
        print(f'after gelu\t\t: {x.size()}')

        x = self.linear_1(x)
        print(f'after layer_1 mlp\t: {x.size()}')

        return x

In [17]:
# Codeblock 16
x = x[:, 0]    #(1)
mlp_head = MLPHead()
x = mlp_head(x)

original		: torch.Size([1, 768])
after norm		: torch.Size([1, 768])
after layer_0 mlp	: torch.Size([1, 768])
after gelu		: torch.Size([1, 768])
after layer_1 mlp	: torch.Size([1, 10])


In [18]:
# Codeblock 17
class ViT(nn.Module):
    def __init__(self):
        super().__init__()

        #self.patcher = PatcherUnfold()
        self.patcher = PatcherConv()    #(1)
        self.pos_embedding = PosEmbedding()
        self.transformer_encoders = nn.Sequential(
            *[TransformerEncoder() for _ in range(NUM_ENCODERS)]    #(2)
            )
        self.mlp_head = MLPHead()

    def forward(self, x):

        x = self.patcher(x)
        x = self.pos_embedding(x)
        x = self.transformer_encoders(x)
        x = x[:, 0]    #(3)
        x = self.mlp_head(x)

        return x

In [19]:
# Codeblock 18
vit = ViT().to(device)
x = torch.randn(1, 3, 224, 224).to(device)
print(vit(x).size())

original		: torch.Size([1, 3, 224, 224])
after conv		: torch.Size([1, 768, 14, 14])
after flatten		: torch.Size([1, 768, 196])
after permute		: torch.Size([1, 196, 768])
class_token dim		: torch.Size([1, 1, 768])
before concat		: torch.Size([1, 196, 768])
after concat		: torch.Size([1, 197, 768])
after pos_embedding	: torch.Size([1, 197, 768])
after dropout		: torch.Size([1, 197, 768])
residual dim		: torch.Size([1, 197, 768])
after norm		: torch.Size([1, 197, 768])
after attention		: torch.Size([1, 197, 768])
after addition		: torch.Size([1, 197, 768])
residual dim		: torch.Size([1, 197, 768])
after norm		: torch.Size([1, 197, 768])
after mlp		: torch.Size([1, 197, 768])
after addition		: torch.Size([1, 197, 768])
residual dim		: torch.Size([1, 197, 768])
after norm		: torch.Size([1, 197, 768])
after attention		: torch.Size([1, 197, 768])
after addition		: torch.Size([1, 197, 768])
residual dim		: torch.Size([1, 197, 768])
after norm		: torch.Size([1, 197, 768])
after mlp		: torch.Siz

In [24]:
# Codeblock 19
summary(vit, input_size=(1,3,224,224))

original		: torch.Size([1, 3, 224, 224])
after conv		: torch.Size([1, 768, 14, 14])
after flatten		: torch.Size([1, 768, 196])
after permute		: torch.Size([1, 196, 768])
class_token dim		: torch.Size([1, 1, 768])
before concat		: torch.Size([1, 196, 768])
after concat		: torch.Size([1, 197, 768])
after pos_embedding	: torch.Size([1, 197, 768])
after dropout		: torch.Size([1, 197, 768])
residual dim		: torch.Size([1, 197, 768])
after norm		: torch.Size([1, 197, 768])
after attention		: torch.Size([1, 197, 768])
after addition		: torch.Size([1, 197, 768])
residual dim		: torch.Size([1, 197, 768])
after norm		: torch.Size([1, 197, 768])
after mlp		: torch.Size([1, 197, 768])
after addition		: torch.Size([1, 197, 768])
residual dim		: torch.Size([1, 197, 768])
after norm		: torch.Size([1, 197, 768])
after attention		: torch.Size([1, 197, 768])
after addition		: torch.Size([1, 197, 768])
residual dim		: torch.Size([1, 197, 768])
after norm		: torch.Size([1, 197, 768])
after mlp		: torch.Siz

Layer (type:depth-idx)                   Output Shape              Param #
ViT                                      [1, 10]                   --
├─PatcherConv: 1-1                       [1, 196, 768]             --
│    └─Conv2d: 2-1                       [1, 768, 14, 14]          590,592
│    └─Flatten: 2-2                      [1, 768, 196]             --
├─PosEmbedding: 1-2                      [1, 197, 768]             152,064
│    └─Dropout: 2-3                      [1, 197, 768]             --
├─Sequential: 1-3                        [1, 197, 768]             --
│    └─TransformerEncoder: 2-4           [1, 197, 768]             --
│    │    └─LayerNorm: 3-1               [1, 197, 768]             1,536
│    │    └─MultiheadAttention: 3-2      [1, 197, 768]             2,362,368
│    │    └─LayerNorm: 3-3               [1, 197, 768]             1,536
│    │    └─Sequential: 3-4              [1, 197, 768]             4,722,432
│    └─TransformerEncoder: 2-5           [1, 197, 768] 